# 足迹功能实验报告

## 一、数据表结构
```sql
CREATE TABLE footprints (
    footprint_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    content TEXT,
    image_url TEXT,
    created_at DATETIME NOT NULL,
    user_id INTEGER NOT NULL REFERENCES users(user_id),
    location_id INTEGER NOT NULL REFERENCES locations(location_id)
)
```

## 二、核心接口说明

### 1. 创建足迹
```python
def create_footprint(self, user_id, title, content, location_id):
    with self._get_connection() as conn:
        cursor = conn.cursor()
        try:
            cursor.execute('''
                INSERT INTO footprints 
                (title, content, image_url, created_at, user_id, location_id)
                VALUES (?, ?, ?, datetime('now'), ?, ?)
            ''', (title, content, f'img_{int(time.time())}.jpg', user_id, location_id))
            conn.commit()
            return cursor.lastrowid
        except sqlite3.Error as e:
            print(f"创建足迹失败: {str(e)}")
            return None
```

#### 参数说明：
| 参数名       | 类型   | 必填 | 说明                |
|--------------|--------|------|---------------------|
| user_id      | int    | 是   | 用户ID（外键约束） |
| title        | str    | 是   | 足迹标题            |
| content      | str    | 否   | 图文内容            |
| location_id  | int    | 是   | 地点ID（外键约束） |

---

### 2. 多条件查询
```python
def get_footprints_by_filters(self, username=None, location_name=None, 
                            location_types=None, created_after=None, 
                            created_before=None):
```

#### 查询条件示例：
```python
# 查询用户名为"test"、地点类型为景点、创建时间在2024年后的足迹
db.get_footprints_by_filters(
    username="test",
    location_types=["attraction"],
    created_after="2024-01-01 00:00:00"
)
```

#### 返回数据结构：
```json
[
    {
        "footprint_id": 1,
        "title": "长城游记",
        "content": "雄伟的古代防御工程",
        "image_url": "img_1620000000.jpg",
        "created_at": "2024-01-20 14:30",
        "user_id": 1,
        "location_id": 1,
        "location_name": "Great Wall",
        "location_type": "attraction",
        "username": "test_user"
    }
]
```

### 3. 足迹详情查询
```python
def get_footprint_detail(self, footprint_id):
    with self._get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT f.*, l.name as location_name, u.username 
            FROM footprints f
            JOIN locations l ON f.location_id = l.location_id
            JOIN users u ON f.user_id = u.user_id
            WHERE f.footprint_id = ?
        ''', (footprint_id,))
        row = cursor.fetchone()
        if row:
            return {
                'footprint_id': row[0],
                'title': row[1],
                'content': row[2],
                'image_url': row[3],
                'created_at': row[4],
                'user_id': row[5],
                'location_id': row[6],
                'location_name': row[7],
                'username': row[8]
            }
        return None
```

#### 功能特性
| 特性	| 说明 |
|-------|-------|
|精确查询 | 通过footprint_id精准定位单条足迹记录|
|关联数据获取 | 自动获取关联的用户名和地点名称 |
|空值处理 | 当ID不存在时返回None |

#### 请求示例
```python
# 查询ID为123的足迹详情
detail = db.get_footprint_detail(123)
print(detail['location_name'])  # 输出"Great Wall"
```

### 4. 足迹信息更新
```python
def update_footprint(self, footprint_id, title=None, content=None, location_id=None):
    with self._get_connection() as conn:
        cursor = conn.cursor()
        try:
            update_fields = []
            params = []
            
            if title is not None:
                update_fields.append("title = ?")
                params.append(title)
            if content is not None:
                update_fields.append("content = ?")
                params.append(content)
            if location_id is not None:
                update_fields.append("location_id = ?")
                params.append(location_id)
                
            if not update_fields:
                raise ValueError("至少需要提供一个更新字段")
                
            params.append(footprint_id)
            query = f'''
                UPDATE footprints SET
                {', '.join(update_fields)}
                WHERE footprint_id = ?
            '''
            cursor.execute(query, params)
            conn.commit()
            return cursor.rowcount > 0
        except sqlite3.Error as e:
            conn.rollback()
            print(f"更新失败: {str(e)}")
            return False
```

#### 更新规则
| 参数 | 类型 | 约束 |
|------|------|------|
|footprint_id|int|必填，目标足迹ID|
|title|str|可选，新标题（非空）|
|content|str|可选，新内容（允许空串）|
|location_id|str|可选，有效地点ID|


# 地点功能实验报告

## 一、数据表结构
```sql
-- 主地点表
CREATE TABLE locations (
    location_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    address TEXT,
    type TEXT CHECK(type IN ('attraction','restaurant','transport'))
)

-- 行程-地点关联表
CREATE TABLE trip_locations (
    location_id INTEGER NOT NULL REFERENCES locations(location_id) ON DELETE CASCADE,
    trip_id INTEGER NOT NULL REFERENCES trips(trip_id) ON DELETE CASCADE, 
    PRIMARY KEY (location_id, trip_id)
```

## 二、核心接口说明

### 1. 创建地点
```python
def create_location(self, name, address, location_type):
    with self._get_connection() as conn:
        cursor = conn.cursor()
        try:
            valid_types = {'attraction','restaurant','transport'}
            if location_type not in valid_types:
                raise ValueError(f"Invalid type: {location_type}")
            cursor.execute('''
                INSERT INTO locations 
                (name, address, type)
                VALUES (?, ?, ?)
            ''', (name, address, location_type))
            return cursor.lastrowid
```

#### 参数说明：
| 参数名         | 类型   | 必填 | 说明                         |
|----------------|--------|------|------------------------------|
| name           | str    | 是   | 地点名称（唯一约束）         |
| address        | str    | 否   | 详细地址信息                 |
| location_type  | str    | 是   | attraction/restaurant/transport |

#### 异常处理：
```python
try:
    db.create_location("故宫", "北京东城区", "attraction")
except sqlite3.IntegrityError:
    print("地点名称已存在")  # 触发UNIQUE约束
except ValueError:
    print("类型不合法")     # 触发CHECK约束
```

---

### 2. 行程地点绑定（在create_trip中实现）
```python
def create_trip(self, participants, start_day, end_day, location_ids):
    if location_ids:
        unique_locations = list(set(location_ids))
        cursor.execute(f'''
            SELECT location_id FROM locations
            WHERE location_id IN ({','.join(['?']*len(unique_locations))})
        ''', unique_locations)
        valid_locations = [row[0] for row in cursor.fetchall()]
        
        cursor.executemany('''
            INSERT INTO trip_locations VALUES (?, ?)
        ''', [(lid, trip_id) for lid in valid_locations])
```

#### 绑定规则：
| 规则类型         | SQL实现                      |
|------------------|------------------------------|
| 级联删除         | ON DELETE CASCADE            |
| 主键约束         | PRIMARY KEY(location_id, trip_id) |

---

### 3. 全量地点查询
```python
def get_all_locations(self):
    cursor.execute('''
        SELECT location_id, name, type 
        FROM locations
    ''')
    return [{
        'location_id': row[0],
        'name': row[1],
        'type': row[2]
    } for row in cursor.fetchall()]
```

#### 返回示例：
```json
{
    "location_id": 8,
    "name": "Great Wall",
    "type": "attraction"
}
```
```